2021.08.09 OIE疫情資料爬蟲

In [1]:
import pandas as pd
import requests
from datetime import datetime, timedelta

- 目標網站:
    - https://wahis.oie.int/pi/getLatestReports (最新版的JS，GET)
    - https://wahis.oie.int/pi/getReportList (詳細版的JS，POST)

In [2]:
# url =  "https://wahis.oie.int/pi/getLatestReports"
url_all = 'https://wahis.oie.int/pi/getReportList'

# 最新版

In [3]:
# df = pd.read_json(url)
# df

# 詳細版

In [4]:
rq_data = {
  "pageNumber":1,
  "pageSize":500,
  "searchText":"",
  "sortColName":"",
  "sortColOrder":"ASC",
  "reportFilters":{"reportDate":{"startDate":(datetime.today() + timedelta(days=-14)).strftime("%Y-%m-%d"),
                                 "endDate":datetime.today().strftime("%Y-%m-%d")}},
  "languageChanged":"false"
}

In [5]:
rq_data # 近2週

{'pageNumber': 1,
 'pageSize': 500,
 'searchText': '',
 'sortColName': '',
 'sortColOrder': 'ASC',
 'reportFilters': {'reportDate': {'startDate': '2021-08-10',
   'endDate': '2021-08-24'}},
 'languageChanged': 'false'}

In [6]:
r = requests.post(url_all, json=rq_data) # POST內容有nested json 要用參數json= 而非 data=
df_all = pd.DataFrame(r.json()["homePageDto"])
df_all.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 196 entries, 0 to 195
Data columns (total 17 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   country              196 non-null    object
 1   reportId             196 non-null    int64 
 2   status               196 non-null    object
 3   diseases             196 non-null    object
 4   reason               196 non-null    object
 5   eventDate            196 non-null    object
 6   reportDate           196 non-null    object
 7   reportType           196 non-null    object
 8   reportInfoId         196 non-null    int64 
 9   diseaseType          98 non-null     object
 10  eventStatus          196 non-null    object
 11  flagTranslated       196 non-null    bool  
 12  region               196 non-null    object
 13  reportNumber         196 non-null    int64 
 14  reportHistoryType    196 non-null    object
 15  eventIdOIEReference  196 non-null    object
 16  reportRe

In [7]:
df_all.head()

,country,reportId,status,diseases,reason,eventDate,reportDate,reportType,reportInfoId,diseaseType,eventStatus,flagTranslated,region,reportNumber,reportHistoryType,eventIdOIEReference,reportReference
0,Latvia,151067,Validated,Highly pathogenic influenza A viruses (Inf. wi...,First occurrence in the country,2021-02-09,2021-08-10,FUR,37377,H5N8,On-going,True,Europe,0,Follow-up report22,38161,NaN
1,Latvia,151068,Validated,Highly pathogenic influenza A viruses (Inf. wi...,New strain in the country,2021-06-02,2021-08-10,FUR,37378,H5N1,On-going,True,Europe,0,Follow-up report9,evt_3748,NaN
2,Latvia,151069,Validated,SARS-CoV-2 in animals (Inf. with),Emerging disease,2021-04-09,2021-08-10,FUR,37379,NaN,On-going,True,Europe,0,Follow-up report15,evt_3687,NaN
3,Latvia,151070,Validated,African swine fever virus (Inf. with),Recurrence,2021-01-01,2021-08-10,FUR,37381,NaN,On-going,True,Europe,0,Follow-up report28,37503,NaN
4,Korea (Rep. of),151055,Validated,Acute hepatopancreatic necrosis disease (2016-),Recurrence,2021-04-06,2021-08-10,FUR,37406,NaN,On-going,True,Asia,0,Follow-up report1,evt_3691,


## 挑選重點疫情

- 挑選邏輯:
    - diseases 文字敘述中有 Highly pathogenic
    - diseaseType 有值(有找到"H")
    - status 為 Validated


In [8]:
mask = (df_all["diseases"].str.contains('Highly pathogenic', regex=False).fillna(False)) \
            & (df_all["diseaseType"].str.contains('H', regex=False).fillna(False)) \
            & (df_all["status"].isin(["Validated"]))
df_all_pic = df_all.loc[mask,:]
df_all_pic.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 85 entries, 0 to 183
Data columns (total 17 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   country              85 non-null     object
 1   reportId             85 non-null     int64 
 2   status               85 non-null     object
 3   diseases             85 non-null     object
 4   reason               85 non-null     object
 5   eventDate            85 non-null     object
 6   reportDate           85 non-null     object
 7   reportType           85 non-null     object
 8   reportInfoId         85 non-null     int64 
 9   diseaseType          85 non-null     object
 10  eventStatus          85 non-null     object
 11  flagTranslated       85 non-null     bool  
 12  region               85 non-null     object
 13  reportNumber         85 non-null     int64 
 14  reportHistoryType    85 non-null     object
 15  eventIdOIEReference  85 non-null     object
 16  reportRef

## 疫情統整

In [9]:
# 以 reportDate、country 為 group， 統計疫情數量、report_url 多筆取締一筆當代表
gp_cols = ["reportDate","country","diseaseType"]
df_report = df_all_pic[gp_cols+["reportId"]].groupby(gp_cols).agg(['count', 'first']).reset_index()
df_report.columns = gp_cols + ["report_count","report_url"]
df_report.sort_values(by=["reportDate"], ascending=[0], inplace=True)
df_report.reset_index(drop=True, inplace=True)
df_report.loc[:,"report_url"] = df_report.loc[:,"report_url"].apply(lambda _id: f"https://wahis.oie.int/#/report-info?reportId={_id}")
df_report

,reportDate,country,diseaseType,report_count,report_url
0,2021-08-23,Vietnam,H5N6,1,https://wahis.oie.int/#/report-info?reportId=1...
1,2021-08-22,Vietnam,H5N8,1,https://wahis.oie.int/#/report-info?reportId=1...
2,2021-08-21,Vietnam,H5N6,1,https://wahis.oie.int/#/report-info?reportId=1...
3,2021-08-20,United Kingdom,H5N8,2,https://wahis.oie.int/#/report-info?reportId=1...
4,2021-08-20,United Kingdom,H5N5,1,https://wahis.oie.int/#/report-info?reportId=1...
5,2021-08-20,United Kingdom,H5N3,1,https://wahis.oie.int/#/report-info?reportId=1...
6,2021-08-20,United Kingdom,H5N1,2,https://wahis.oie.int/#/report-info?reportId=1...
7,2021-08-20,Togo,H5N1,2,https://wahis.oie.int/#/report-info?reportId=1...
8,2021-08-20,Cote D'Ivoire,H5N1,1,https://wahis.oie.int/#/report-info?reportId=1...
9,2021-08-20,Benin,H5,1,https://wahis.oie.int/#/report-info?reportId=1...


## 欄位名稱對齊資料庫

In [10]:
# OIE 原始資料 範例
# {
#     "RelatedReports": "NULL",
#     "CausalAgent": "HPAI",
#     "DateOfConfirmationOfTheEvent": "14/02/2020",
#     "ManifestationOfDisease": "Clinical disease",
#     "DateEventResolved": "NULL",
#     "ReasonForNotification": "Recurrence of a listed disease",
#     "ThisEventPertainsTo": "A Defined Zone Within The Country",
#     "outbreaks": "2",
#     "NatureOfDiagnosis": "Clinical, Laboratory (advanced)",
#     "DateOfPreviousOccurrence": "04/06/2019",
#     "DateSubmittedToOie": "2020-02-17",
#     "Serotype": "H5N1",
#     "ReportType": "Immediate notification",
#     "lat": null,
#     "_id": "33305",
#     "description": "Sourced from OIE website.",
#     "DateOfStartOfTheEvent": "11/02/2020",
#     "area": "Vietnam",
#     "lon": null,
#     "ReportDate": "17/02/2020",
#     "url": "https://www.oie.int/wahis_2/public/wahid.php/Reviewreport/Review?page_refer=MapFullEventReport&reportid=33305"
# }

In [11]:
df_report.head(1).to_dict()

{'reportDate': {0: '2021-08-23'},
 'country': {0: 'Vietnam'},
 'diseaseType': {0: 'H5N6'},
 'report_count': {0: 1},
 'report_url': {0: 'https://wahis.oie.int/#/report-info?reportId=151311'}}

In [13]:
# 重新命名
col_rename_dict = {
 'reportDate': "ReportDate",
 'country': "area",
 'diseaseType': "Serotype",
 'report_count': "outbreaks",
 "report_url" : "url"
}
df_report.rename(columns=col_rename_dict, inplace=True)

# 還原欄位(舊資料有，新資料沒有，但對功能沒有意義)
rebuildColsLst = ["RelatedReports", "DateOfConfirmationOfTheEvent", "ManifestationOfDisease", "DateEventResolved",
                 "ReasonForNotification", "ThisEventPertainsTo", "NatureOfDiagnosis", "DateOfPreviousOccurrence",
                 "DateSubmittedToOie", "ReportType", "lat", "description", "DateOfStartOfTheEvent","lon"]
for _c in rebuildColsLst:
    df_report.loc[:, _c] = None


# 創建欄位(舊資料有，新資料沒有，對功能有意義)
df_report.loc[:, "_id"] = df_report.loc[:, "ReportDate"] + "_" + df_report.loc[:, "area"]
df_report.loc[:, "CausalAgent"] = "HPAI"

# 對齊欄位格式
df_report.loc[:, "ReportDate"] = df_report.loc[:, "ReportDate"].apply(lambda s: "/".join([s.split("-")[2],s.split("-")[1],s.split("-")[0]]) 
                                                                         if isinstance(s, str) else "1/1/2000")


In [14]:
df_report.head()

,ReportDate,area,Serotype,outbreaks,url,RelatedReports,DateOfConfirmationOfTheEvent,ManifestationOfDisease,DateEventResolved,ReasonForNotification,...,NatureOfDiagnosis,DateOfPreviousOccurrence,DateSubmittedToOie,ReportType,lat,description,DateOfStartOfTheEvent,lon,_id,CausalAgent
0,23/08/2021,Vietnam,H5N6,1,https://wahis.oie.int/#/report-info?reportId=1...,None,None,None,None,None,...,None,None,None,None,None,None,None,None,2021-08-23_Vietnam,HPAI
1,22/08/2021,Vietnam,H5N8,1,https://wahis.oie.int/#/report-info?reportId=1...,None,None,None,None,None,...,None,None,None,None,None,None,None,None,2021-08-22_Vietnam,HPAI
2,21/08/2021,Vietnam,H5N6,1,https://wahis.oie.int/#/report-info?reportId=1...,None,None,None,None,None,...,None,None,None,None,None,None,None,None,2021-08-21_Vietnam,HPAI
3,20/08/2021,United Kingdom,H5N8,2,https://wahis.oie.int/#/report-info?reportId=1...,None,None,None,None,None,...,None,None,None,None,None,None,None,None,2021-08-20_United Kingdom,HPAI
4,20/08/2021,United Kingdom,H5N5,1,https://wahis.oie.int/#/report-info?reportId=1...,None,None,None,None,None,...,None,None,None,None,None,None,None,None,2021-08-20_United Kingdom,HPAI


# 儲存

In [15]:
df_report.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58 entries, 0 to 57
Data columns (total 21 columns):
 #   Column                        Non-Null Count  Dtype 
---  ------                        --------------  ----- 
 0   ReportDate                    58 non-null     object
 1   area                          58 non-null     object
 2   Serotype                      58 non-null     object
 3   outbreaks                     58 non-null     int64 
 4   url                           58 non-null     object
 5   RelatedReports                0 non-null      object
 6   DateOfConfirmationOfTheEvent  0 non-null      object
 7   ManifestationOfDisease        0 non-null      object
 8   DateEventResolved             0 non-null      object
 9   ReasonForNotification         0 non-null      object
 10  ThisEventPertainsTo           0 non-null      object
 11  NatureOfDiagnosis             0 non-null      object
 12  DateOfPreviousOccurrence      0 non-null      object
 13  DateSubmittedToOie    